In [130]:
using SimJulia

#### How to chose if only one event needs to be realised
##### Example setting: 
an agent requests a resource but only has a limited amount of patience before no longer wanting/needing the resource. 
```Julia
@resumable function agent(sim::Simulation,con::Resource)
    req = request(con)
    res = @yield req | timeout(sim,4)
    if res[req].state == SimJulia.processed
        println("using my resource...")
        @yield timeout(sim,1)
        release(res)
    else
        println("patience ran out... on time $(sim.time)")
        cancel(con,req)
    end
end

sim = Simulation()
con = Resource(sim,0)
@process agent(sim,con)
run(sim,10)
```
`patience ran out... on time 4.0`

For the example, a simulation is made with a `::Resource` with a capacity of $0$. So the agent can never obtain the requested resource. In the `agent` function the following happens:
1. A request for `con::Resource` is made. The type of `req` is `SimJulia.Put`. This event will be triggered by an `@yield`
2. the variable `req` is a dictionary with the events as key and the `::StateValue` as value. The first event to have been processed will have its `::StateValue` equal to `SimJulia.processed`
3. the `if` conditions test whether the `::StateValue` of our request is equal to `SimJulia.processed`. 
  1. If this is the case, the agent obtains the `::Resource`, uses it for 1 time unit and releases it back for further use.
  2. If this is NOT the case, the other event will have taken place (in this case the timeout) and we remove the request from the `::Resource` queue with `cancel`.
4. the simulation terminates since no more processes are active on time 4.0.

```
req: SimJulia.Put 4

res: Dict{AbstractEvent,SimJulia.StateValue}(SimJulia.Timeout 5=>StateValue(processed, nothing),
SimJulia.Put 4=>StateValue(idle, nothing))```

The above method can also be used for more than two events where the completion of a single one of them is sufficient.

#### How to chose between whatever resource comes available first
Setting: when dealing with multiple available resources, it might be sufficient for an agent to obtain a single one of them in order to be able to proceed. In the example below this is illustrated for two different resources. The difference between both is the time required before they become available again. One of both resource has two units while the other only has one. Several agents (requiring just one of both resources to function) are active.
```Julia
@resumable function agent(sim::Simulation,res1::Resource,res2::Resource,name::String)
    r1 = request(res1)
    r2 = request(res2)
    res = @yield r1 | r2
    if res[r1].state == SimJulia.processed
        println("$name obtains slow resource on time $(sim.time)")
        if res[r2].state == SimJulia.processed
            release(res2)
        else
            cancel(res2,r2)
        end
        @yield timeout(sim,2)
        release(res1)
        println("$name releases slow resource on time $(sim.time)")
    else
        cancel(res1,r1)
        println("$name obtains fast resource on time $(sim.time)")
        @yield timeout(sim,1)
        release(res2)
        println("$name releases fast resource on time $(sim.time)")
    end
end

sim = Simulation()
slowresource = Resource(sim,1)
fastresource = Resource(sim,2)
for i in 1:7
    @process agent(sim,slowresource,fastresource,"Agent $i")
end
run(sim)
```

```
Agent 1 obtains slow resource on time 0.0
Agent 2 obtains fast resource on time 0.0
Agent 3 obtains fast resource on time 0.0
Agent 2 releases fast resource on time 1.0
Agent 3 releases fast resource on time 1.0
Agent 4 obtains fast resource on time 1.0
Agent 5 obtains fast resource on time 1.0
Agent 1 releases slow resource on time 2.0
Agent 4 releases fast resource on time 2.0
Agent 5 releases fast resource on time 2.0
Agent 6 obtains slow resource on time 2.0
Agent 7 obtains fast resource on time 2.0
Agent 7 releases fast resource on time 3.0
Agent 6 releases slow resource on time 4.0
```
In the agent the following happens:
1. Individual requests for both resources `res1` and `res2` are made. The type of `r1`/`r2` is `SimJulia.Put`. The events will be triggered by an `@yield`
2. the variable `res` is a dictionary with the events as key and the `::StateValue` as value. The first event to have been processed will have its `::StateValue` equal to `SimJulia.processed`
3. the `if` conditions test whether the `::StateValue` of our request for resource `res1` is equal to `SimJulia.processed`. 
  1. If this is the case, the agent obtains the `::Resource`. Meanwhile is it possible that `res2` was also available at the same time. In order to avoid double resource usage we need to do one of both things:
    * release `res2` if it was obtained at the sime time as `res1`
    * remove the request for `res2` from its queue with `cancel`
  
    In the example `res1` is consided as the slow resource. The agent uses it for 2 time units and releases it back for further use.
  2. If this is NOT the case, we will have obtained `res2`. Again, to avoid double resource usage, we remove the request for `res1` from its queue with `cancel`. The agent uses it for 1 time unit and releases `res2` back for further use.
4. the simulation terminates when all agents have been processed.

#### Competing for a Resource
When to entities request a `::Resource` at the same time, you can determine which one should get priority base on the `priority` keyword in the `request` instruction. The lower the integer value, the higher the priority. Default values for all request are equal to 0.

```Julia
using SimJulia

@resumable function grabsource(sim::Simulation,res::Resource)
    for i in 1:4
        @yield request(res)
    end
    println("got them all on time $(sim.time)")
    @yield timeout(sim,10)
    @yield release(res)
    println("released ONE on time $(sim.time)")
end

@resumable function priosource(sim::Simulation,res::Resource)
    @yield timeout(sim,1)
    @yield request(res,priority = -1)
    println("PRIO got it at $(sim.time)")
    @yield timeout(sim,1)
    @yield release(res,priority = -1)
end

@resumable function secondsource(sim::Simulation,res::Resource)
    @yield timeout(sim,1)
    @yield request(res,priority = 1)
    println("SECOND got it at $(sim.time)")
    @yield timeout(sim,1)
    @yield release(res)
end

sim = Simulation()
res = Resource(sim,4)
@process grabsource(sim, res)
@process priosource(sim, res)
@process secondsource(sim,res)
run(sim,20)
```
```
got them all on time 0.0
released ONE on time 10.0
PRIO got it at 10.0
SECOND got it at 11.0
```

1. Initialy (in time 0) all `Resources` are attributed to the grabsource process.
2. On time one, both the priosource and secondsource process request a resource. The priority of priosource however is a lower number and gets a higher priority.
2. On time 10 the grabsource process releases one `Resources`. At the sime time, the `Resource`is attributed to the priosource process.

